In [2]:
import lumipy as lm
import time
from lusidjam import RefreshingToken as rt

# Tutorial 1: Client

Lumipy provides a client class for interfacing to the Luminesce web API and getting results of queries back as pandas DataFrames. Any query you would normally send with the CLI or web GUI will work in the same way here when you pass in a query string. The methods on the client match those on the web api with a couple of additions for convenience.  

## Authentication

Credentials can be supplied as a secrets file or as environment variables (for example, when working in a containerised job) or by supplying a valid token directly (e.g. `RefreshingToken()` in Jupyterhub).

The secrets file is expected to have the following form
```
{
    "api" : {
      "tokenUrl": "<auth token url>",
      "username": "<user name>",
      "password": "<user password>",
      "clientId": "<client id>",
      "clientSecret": "<client secret>",
      "lumiApiUrl": "https://<client>.lusid.com/honeycomb/api/"
    }
}
```

## Build the Client Object

If no path or token is supplied to the constructor the client will fall back on trying to find authentication information in the available environment variables. 

In [2]:
client = lm.get_client(token=rt())

## Sending Queries to Luminesce

The client can be used to send SQL query strings to luminesce either synchronously or asynchronously. 

### Define Query in String

In [3]:
qry_str = """
select 
    ^ 
from 
    Lusid.Instrument
limit 
    5
"""

### Run Synchronously

In [4]:
df = client.query_and_fetch(qry_str)
df

,LusidInstrumentId,ClientInternal,Quasar,ThirdpartyTicker,NTSecurityNumber,Figi,Cins,SecurityNumber,SecurityUniqueQualifier,InstrumentId,AssetId,BloombergGlobalIndentifier,BloombergUniqueId,DisplayName,IsActive
0,LUID_00AC37Z6,3021357-DESSE-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KO CALL (Philips Electric) B 01/01/9999 EUR 9....,True
1,LUID_005PR4LM,5599271-AUBND-AUD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZC CD REDEEM 05/10/2018 AUD - Ser 01,True
2,LUID_0025ALY6,4757553-HKSEHK-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW CALL (China Life Ins) E 23/12/2016 HKD 21.33,True
3,LUID_006X1WHZ,4776560-THSET-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW PUT (PTT Global Chemical) E 03/11/2016 THB ...,True
4,LUID_00497N27,971126-USNYSE-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.25% NT REDEEM 15/01/2016 USD 2000,True


### Run Asynchronously

Start the query and get store the execution ID you get back.

In [5]:
ex_id = client.start_query(qry_str)
print(f'Query started with execution id = {ex_id}')

Query started with execution id = 5a43134c-afcd-473e-a2e7-6d25f398eda6


Check the query's status. This returns the JSON formatted status response as a dictionary.

In [6]:
status = client.get_status(ex_id)
while not status['status'] == 'RanToCompletion':
    status = client.get_status(ex_id)
    time.sleep(1)

Get the data from the completed query using the execution ID. 

In [7]:
df = client.get_result(ex_id)
df

Fetching data... 📡
  Page   0 downloaded (5 rows).
Data fetch finished


,LusidInstrumentId,ClientInternal,Quasar,ThirdpartyTicker,NTSecurityNumber,Figi,Cins,SecurityNumber,SecurityUniqueQualifier,InstrumentId,AssetId,BloombergGlobalIndentifier,BloombergUniqueId,DisplayName,IsActive
0,LUID_00K5SR3C,359763-LULSE-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SU NT REDEEM 15/05/2011 USD 5000 - Ser `I098`,True
1,LUID_00K6J679,955741--USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.25% BD IS 15/05/2030 USD 36.25 US31359MFP32,True
2,LUID_00GDKG7N,4161588-TWBND-TWD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZC TBILL REDEEM 26/12/2014 TWD 10000,True
3,LUID_00GFCZ99,5355524-USTRCE-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZC MTN REDEEM 27/12/2024 USD,True
4,LUID_00KLI3GN,1328074--USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FR NT REDEEM 15/07/2010 USD,True


## Catalogs

### Table and Field 

In [8]:
client.table_field_catalog()

,TableName,FieldName,DataType,FieldType,IsPrimaryKey,IsMain,Description,ParamDefaultValue,TableParamColumns,ProviderDescription
0,Consolidatedtapeprovider.Posttrade,InstrumentId,Text,Column,False,True,None,None,None,Database Query 'Consolidatedtapeprovider.Postt...
1,Consolidatedtapeprovider.Posttrade,InstrumentIdType,Text,Column,False,True,None,None,None,Database Query 'Consolidatedtapeprovider.Postt...
2,Consolidatedtapeprovider.Posttrade,NotionalAmount,Decimal,Column,False,True,None,None,None,Database Query 'Consolidatedtapeprovider.Postt...
3,Consolidatedtapeprovider.Posttrade,NotionalCurrency,Text,Column,False,True,None,None,None,Database Query 'Consolidatedtapeprovider.Postt...
4,Consolidatedtapeprovider.Posttrade,Price,Decimal,Column,False,True,None,None,None,Database Query 'Consolidatedtapeprovider.Postt...
...,...,...,...,...,...,...,...,...,...,...
6728,Vendordatastore.Firds.Instrument.Writer,InstrumentName,Text,Column,False,False,None,None,None,Database Query 'Vendordatastore.Firds.Instrume...
6729,Vendordatastore.Firds.Instrument.Writer,Isin,Text,Column,False,False,None,None,None,Database Query 'Vendordatastore.Firds.Instrume...
6730,Vendordatastore.Firds.Instrument.Writer,IssuerId,Text,Column,False,False,None,None,None,Database Query 'Vendordatastore.Firds.Instrume...
6731,Vendordatastore.Firds.Instrument.Writer,ToWrite,Table,Parameter,None,None,None,None,Cfi (Text)\r\nIsin (Text)\r\nIssuerId (Text)\r...,Database Query 'Vendordatastore.Firds.Instrume...


## History

In [9]:
ex_id = client.start_history_query()
print(ex_id)

660ca9a2-aa6f-4074-9245-e947f007abe7


In [10]:
status = client.get_history_status(ex_id)
print(status['status'])

WaitingForActivation


In [12]:
client.get_history_result(ex_id).iloc[:, :4]

,At,FinishedAt,QueryName,ExecutionId
0,2021-06-14T10:34:06.7504157,2021-06-14 10:34:12.6319737Z,Query,5a43134c-afcd-473e-a2e7-6d25f398eda6
1,2021-06-14T10:33:54.6268142,2021-06-14 10:34:05.4048866Z,Query,aa2f49e1-686a-4ffe-bc4b-5b6a5aaf1fb4
2,2021-06-14T09:54:31.8087621,2021-06-14 09:54:32.5688227Z,Query,d1a4eec4-4f32-4b9b-884c-b90da79abbf7
3,2021-06-14T09:54:28.7152817,2021-06-14 09:54:30.0148856Z,Query,788e4c11-f2ea-4b25-9b90-5277fdbf474b
4,2021-06-14T09:54:26.0080915,2021-06-14 09:54:27.8609979Z,Query,dad34a24-6338-4f80-ae66-09eee308c59e
...,...,...,...,...
98,2021-06-13T14:16:24.368311,2021-06-13 14:16:27.6149713Z,Query,4143374a-9106-4a6e-bce0-d80e36696dc3
99,2021-06-13T14:11:48.3817373,2021-06-13 14:11:51.4602824Z,Query,bde395ca-9747-4c18-872e-4ab71e891b8c
100,2021-06-13T14:09:28.3853561,2021-06-13 14:09:31.7202891Z,Query,4b285833-7656-4511-83dc-12424642da9f
101,2021-06-13T15:05:03.3413824,2021-06-13 15:05:03.8601619Z,None,5af7fb5b-7934-412d-880f-873d04c4204b
